In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # Gold Layer - Star Schema Creation
# MAGIC Build dimensional model for analytics

# COMMAND ----------

# Import configuration
import sys
sys.path.append("/Workspace/Users/yahyasanbati.mail@gmail.com/GREEN-IT-DATA-PLATFORM/X002_Databricks")
from config import *

import pandas as pd
import os

print("=" * 70)
print("GOLD LAYER - STAR SCHEMA CREATION")
print("=" * 70)
print(f"\nSource: {SILVER_FILE}")
print(f"Destination: {GOLD_PATH}")

# COMMAND ----------

# Load Silver data
print("\nLoading Silver data...")

if os.path.exists(SILVER_FILE):
    df = pd.read_parquet(SILVER_FILE)
    print(f"Loaded {len(df):,} rows from Silver")
else:
    print(f"ERROR: Silver file not found at {SILVER_FILE}")
    dbutils.notebook.exit("Silver file not found")

# COMMAND ----------

# Display Silver data
print("\nSilver Data Preview:")
display(df.head(10))

print("\nAvailable Columns:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")

# COMMAND ----------

# Create Dimension: DIM_WORKLOAD
print("\nCreating DIM_WORKLOAD...")

if 'WORKLOAD_TYPE' in df.columns:
    dim_workload = df[['WORKLOAD_TYPE']].drop_duplicates().reset_index(drop=True)
    dim_workload['workload_key'] = dim_workload.index + 1
    
    print(f"  Unique workload types: {len(dim_workload)}")
    display(dim_workload)
else:
    print("  ERROR: WORKLOAD_TYPE column not found")
    dim_workload = pd.DataFrame(columns=['workload_key', 'WORKLOAD_TYPE'])

# COMMAND ----------

# Create Dimension: DIM_ENERGY
print("\nCreating DIM_ENERGY...")

energy_cols = ['ENERGY_SOURCE']
if 'IS_RENEWABLE' in df.columns:
    energy_cols.append('IS_RENEWABLE')

dim_energy = df[energy_cols].drop_duplicates().reset_index(drop=True)
dim_energy['energy_key'] = dim_energy.index + 1

print(f"  Unique energy sources: {len(dim_energy)}")
display(dim_energy)

# COMMAND ----------

# Create Dimension: DIM_SECURITY
print("\nCreating DIM_SECURITY...")

if 'SECURITY_LEVEL' in df.columns:
    dim_security = df[['SECURITY_LEVEL']].drop_duplicates().reset_index(drop=True)
    dim_security['security_key'] = dim_security.index + 1
    
    print(f"  Unique security levels: {len(dim_security)}")
    display(dim_security)
else:
    print("  ERROR: SECURITY_LEVEL column not found")
    dim_security = pd.DataFrame(columns=['security_key', 'SECURITY_LEVEL'])

# COMMAND ----------

# Create Dimension: DIM_SCENARIO
print("\nCreating DIM_SCENARIO...")

scenario_cols = ['WORKLOAD_SCENARIO']
if 'SCENARIO_STRATEGY' in df.columns:
    scenario_cols.append('SCENARIO_STRATEGY')

dim_scenario = df[scenario_cols].drop_duplicates().reset_index(drop=True)
dim_scenario['scenario_key'] = dim_scenario.index + 1

print(f"  Unique scenarios: {len(dim_scenario)}")
display(dim_scenario)

# COMMAND ----------

# Create Fact Table
print("\nCreating FACT_GREEN_WORKLOAD...")

# Start with full dataframe
fact = df.copy()

# Join with dimensions to get keys
print("  Joining with dimensions...")

if len(dim_workload) > 0:
    fact = fact.merge(dim_workload, on=['WORKLOAD_TYPE'], how='left')
    print("    - Joined with DIM_WORKLOAD")

if len(dim_energy) > 0:
    fact = fact.merge(dim_energy, on=energy_cols, how='left')
    print("    - Joined with DIM_ENERGY")

if len(dim_security) > 0:
    fact = fact.merge(dim_security, on=['SECURITY_LEVEL'], how='left')
    print("    - Joined with DIM_SECURITY")

if len(dim_scenario) > 0:
    fact = fact.merge(dim_scenario, on=scenario_cols, how='left')
    print("    - Joined with DIM_SCENARIO")

# COMMAND ----------

# Select fact columns
print("\nSelecting fact columns...")

# Required fact columns
fact_columns = [
    'workload_key',
    'energy_key',
    'security_key',
    'scenario_key'
]

# Add measure columns if they exist
measure_columns = [
    'ENERGY_CONSUMPTION_KWH',
    'CARBON_EMISSIONS_KGCO2',
    'OPERATIONAL_COST_USD',
    'CARBON_INTENSITY',
    'COST_PER_KWH'
]

for col in measure_columns:
    if col in fact.columns:
        fact_columns.append(col)
        print(f"  Added measure: {col}")

# Create final fact table
fact_green = fact[fact_columns].copy()

# Add fact_key
fact_green.insert(0, 'fact_key', range(1, len(fact_green) + 1))

print(f"\nFact table created:")
print(f"  Rows: {len(fact_green):,}")
print(f"  Columns: {len(fact_green.columns)}")

# COMMAND ----------

# Display fact table sample
print("\nFact Table Sample:")
display(fact_green.head(20))

# COMMAND ----------

# Validate fact table
print("\nFact Table Validation:")
print("=" * 70)

# Check for NULL keys
for key_col in ['workload_key', 'energy_key', 'security_key', 'scenario_key']:
    null_count = fact_green[key_col].isnull().sum()
    if null_count > 0:
        print(f"  WARNING: {key_col} has {null_count} NULL values")
    else:
        print(f"  OK: {key_col} - no NULLs")

print("=" * 70)

# COMMAND ----------

# Save Gold layer
print("\nSaving Gold layer...")

os.makedirs(GOLD_PATH, exist_ok=True)

# Save dimensions
try:
    dim_workload.to_parquet(DIM_WORKLOAD_FILE, index=False)
    print(f"  Saved: DIM_WORKLOAD ({len(dim_workload)} rows)")
except Exception as e:
    print(f"  ERROR saving DIM_WORKLOAD: {e}")

try:
    dim_energy.to_parquet(DIM_ENERGY_FILE, index=False)
    print(f"  Saved: DIM_ENERGY ({len(dim_energy)} rows)")
except Exception as e:
    print(f"  ERROR saving DIM_ENERGY: {e}")

try:
    dim_security.to_parquet(DIM_SECURITY_FILE, index=False)
    print(f"  Saved: DIM_SECURITY ({len(dim_security)} rows)")
except Exception as e:
    print(f"  ERROR saving DIM_SECURITY: {e}")

try:
    dim_scenario.to_parquet(DIM_SCENARIO_FILE, index=False)
    print(f"  Saved: DIM_SCENARIO ({len(dim_scenario)} rows)")
except Exception as e:
    print(f"  ERROR saving DIM_SCENARIO: {e}")

# Save fact
try:
    fact_green.to_parquet(FACT_GREEN_WORKLOAD_FILE, index=False)
    print(f"  Saved: FACT_GREEN_WORKLOAD ({len(fact_green):,} rows)")
except Exception as e:
    print(f"  ERROR saving FACT_GREEN_WORKLOAD: {e}")

# COMMAND ----------

# Display all dimensions
print("\nDimension Tables:")
print("=" * 70)

print("\nDIM_WORKLOAD:")
display(dim_workload)

print("\nDIM_ENERGY:")
display(dim_energy)

print("\nDIM_SECURITY:")
display(dim_security)

print("\nDIM_SCENARIO:")
display(dim_scenario)

# COMMAND ----------

# Gold validation summary
print("\n" + "=" * 70)
print("GOLD STAR SCHEMA SUMMARY")
print("=" * 70)

print(f"\nDimension Tables:")
print(f"  - DIM_WORKLOAD: {len(dim_workload)} rows")
print(f"  - DIM_ENERGY: {len(dim_energy)} rows")
print(f"  - DIM_SECURITY: {len(dim_security)} rows")
print(f"  - DIM_SCENARIO: {len(dim_scenario)} rows")

print(f"\nFact Table:")
print(f"  - FACT_GREEN_WORKLOAD: {len(fact_green):,} rows")

print(f"\nFile Locations:")
print(f"  {DIM_WORKLOAD_FILE}")
print(f"  {DIM_ENERGY_FILE}")
print(f"  {DIM_SECURITY_FILE}")
print(f"  {DIM_SCENARIO_FILE}")
print(f"  {FACT_GREEN_WORKLOAD_FILE}")

print("\nStar Schema Relationships:")
print("  FACT_GREEN_WORKLOAD")
print("    |-- workload_key --> DIM_WORKLOAD")
print("    |-- energy_key --> DIM_ENERGY")
print("    |-- security_key --> DIM_SECURITY")
print("    |-- scenario_key --> DIM_SCENARIO")

print("\n" + "=" * 70)
print("GOLD LAYER COMPLETE")
print("=" * 70)

# COMMAND ----------

# MAGIC %md
# MAGIC ## Gold Layer Complete
# MAGIC 
# MAGIC Star schema successfully created with:
# MAGIC - 4 Dimension tables
# MAGIC - 1 Fact table
# MAGIC 
# MAGIC Ready for:
# MAGIC - Power BI connection
# MAGIC - Tableau dashboards
# MAGIC - SQL analytics
# MAGIC 
# MAGIC All data is stored in M_03_Gold folder